## Concrete scheduling

In [1]:
import docplex.mp 
import sys
import cplex
import numpy as np
import pandas as pd
import os
import math

In [5392]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Linux 64bit
* Python version 3.6.9, located at: /usr/bin/python3
* docplex is present, version is (2, 11, 176)
* CPLEX library is present, version is 12.10.0.0, located at: /home/alious/.local/lib/python3.6/site-packages
* pandas is present, version is 1.1.5


In [5393]:
from docplex.mp.model import Model

Paramètes

In [5394]:
# A_2_5_1 C0
Demand = 20
Q=[15,15]
K=2
a = 200
b=250
TL = 5
M = 2000
coord = [(50,50), (49,39),(34,60)]

In [5395]:
# A_2_5_1 C1
Demand = 20
Q=[15,15]
K=2
a = 280	
b= 310
TL = 5
M = 2000
coord = [(50,50), (49,39),(55,40)]

In [5396]:
# A_2_5_1 C2
Demand = 20
Q=[15,15]
K=2
a = 420	
b= 450
TL = 5
M = 2000
coord = [(50,50), (49,39),(65,42)]

In [5397]:
# A_2_5_1 C3
Demand = 45
Q=[15,15]
K=2
a = 280	
b= 380
TL = 5
M = 2000
coord = [(50,50), (49,39),(55,15)]

In [5398]:
# # A_2_5_1 C4
# Demand = 45
# Q=[15,15]
# K=2
# a = 160
# b= 240
# TL = 5
# M = 2000
# coord = [(50,50), (49,39),(54,48)]

In [5399]:
distance

[[0, 12, 5, 5, 5, 0],
 [12, 0, 11, 11, 11, 12],
 [5, 11, 0, 0, 0, 5],
 [5, 11, 0, 0, 0, 5],
 [5, 11, 0, 0, 0, 5],
 [0, 12, 5, 5, 5, 0]]

In [5400]:
Cout

[[0, 23, 23, 23, 0],
 [23, 0, 22, 22, 5],
 [23, 22, 0, 22, 5],
 [23, 22, 22, 0, 5],
 [0, 5, 5, 5, 0]]

In [5401]:
def Cost(i,j,distance,N):
    if j==N-1 and i in L or i==N-1 and j in L:
        return distance[i+1][j+1]
    
    if i==0 and j in L:
        return distance[0][1] + distance[1][j+1]
    
    if j==0 and i in L:
        return distance[0][1] + distance[1][i+1]
    if i==j:
        return 0
    if i in L and j in L:
        return distance[i+1][1] + distance[1][j+1] 
    
    if i==0 and j == N-1 or j==0 and i ==N-1:
        return 0
    
    return distance[i][j]
    

In [5402]:

qmin= min(Q)
nbDel = int(math.ceil(Demand/qmin))
L= range(1,nbDel+1)
Less= range(1,nbDel)
N = nbDel+2
A= list(np.repeat(a,nbDel))
B= list(np.repeat(b,nbDel))
coords = []
coords= coords + coord[:2]
for i in range(nbDel):
    coords.append(coord[2])
coords.append(coord[0])

distance = [[ math.ceil(math.sqrt( math.pow(c1[0]-c2[0],2) + math.pow(c1[1]-c2[1],2))) 
             for c1 in coords ] for c2 in coords]

Cout = [[Cost(i,j,distance,N) for i in range(N)] for j in range(N)]

S = [ list(range(1,N)) if i==0 else  [i+1]+[N-1] if i < N-2 else [N-1] if i == N-2 else [] for i in range(N)  ]

P= [  [] if i ==0 else list(list(np.arange(0,i))) for i in range(N)  ]

In [5403]:
# S=[[1,4], [2,4], [3,4], [4], []]
# P = [[], [0], [1], [2], [0,1, 2, 3]]
S

[[1, 2, 3, 4], [2, 4], [3, 4], [4], []]

Decision variables

In [5404]:
mdl = Model("scheduling")
# Xijk, nodes (i,j) with k
x = mdl.binary_var_cube(N, N,K, lambda ij: "x_%d_%d_%d" %(ij[0], ij[1],ij[2]))
y = mdl.binary_var(name='y')

# x= mdl.binary_var_dict(d,name='x')
#Ci fin de service au noeud i
C = mdl.continuous_var_list(N,lb=0,ub=1440,name="C")
Z = mdl.continuous_var_list(N,lb=0,ub=1440,name="Z")


In [5405]:
nbDel

3

In [5406]:
mdl.add_constraints( mdl.sum(x[i,j,k] for j in S[i]) ==1 for i in [0] for k in range(K))

[docplex.mp.LinearConstraint[](x_0_1_0+x_0_2_0+x_0_3_0+x_0_4_0,EQ,1),
 docplex.mp.LinearConstraint[](x_0_1_1+x_0_2_1+x_0_3_1+x_0_4_1,EQ,1)]

In [5407]:
mdl.add_constraints( mdl.sum(x[i,j,k] for i in P[j]) ==1 for j in [N-1] for k in range(K))

[docplex.mp.LinearConstraint[](x_0_4_0+x_1_4_0+x_2_4_0+x_3_4_0,EQ,1),
 docplex.mp.LinearConstraint[](x_0_4_1+x_1_4_1+x_2_4_1+x_3_4_1,EQ,1)]

In [5408]:
mdl.add_constraints( mdl.sum(x[i,j,k] for j  in S[i] ) ==  mdl.sum(x[j,i,k] for j in P[i] ) for k in range(K) for i in L)

[docplex.mp.LinearConstraint[](x_1_2_0+x_1_4_0,EQ,x_0_1_0),
 docplex.mp.LinearConstraint[](x_2_3_0+x_2_4_0,EQ,x_0_2_0+x_1_2_0),
 docplex.mp.LinearConstraint[](x_3_4_0,EQ,x_0_3_0+x_1_3_0+x_2_3_0),
 docplex.mp.LinearConstraint[](x_1_2_1+x_1_4_1,EQ,x_0_1_1),
 docplex.mp.LinearConstraint[](x_2_3_1+x_2_4_1,EQ,x_0_2_1+x_1_2_1),
 docplex.mp.LinearConstraint[](x_3_4_1,EQ,x_0_3_1+x_1_3_1+x_2_3_1)]

In [5409]:
mdl.add_constraints( [mdl.sum(Q[k]*x[i,j,k] for i in L for k in range(K) for j in S[i]   ) >= Demand*y] )

[docplex.mp.LinearConstraint[](15x_1_2_0+15x_1_2_1+15x_1_4_0+15x_1_4_1+15x_2_3_0+15x_2_3_1+15x_2_4_0+15x_2_4_1+15x_3_4_0+15x_3_4_1,GE,45y)]

In [5410]:
mdl.add_constraints( mdl.sum(x[i,j,k] for j in S[i] for k in range(K)) <= 1 for i in L )


[docplex.mp.LinearConstraint[](x_1_2_0+x_1_2_1+x_1_4_0+x_1_4_1,LE,1),
 docplex.mp.LinearConstraint[](x_2_3_0+x_2_3_1+x_2_4_0+x_2_4_1,LE,1),
 docplex.mp.LinearConstraint[](x_3_4_0+x_3_4_1,LE,1)]

In [5411]:
mdl.add_constraints( mdl.sum(x[i,j,k] for i in P[j] for k in range(K)) <= 1 for j in L )

[docplex.mp.LinearConstraint[](x_0_1_0+x_0_1_1,LE,1),
 docplex.mp.LinearConstraint[](x_0_2_0+x_0_2_1+x_1_2_0+x_1_2_1,LE,1),
 docplex.mp.LinearConstraint[](x_0_3_0+x_0_3_1+x_1_3_0+x_1_3_1+x_2_3_0+x_2_3_1,LE,1)]

In [5412]:
 mdl.add_constraints( mdl.sum(x[i,j,k] for j in S[i] for k in range(K) ) 
                     <= mdl.sum(x[j,i,k] for j in P[i] for k in range(K) ) for i in L )

[docplex.mp.LinearConstraint[](x_1_2_0+x_1_2_1+x_1_4_0+x_1_4_1,LE,x_0_1_0+x_0_1_1),
 docplex.mp.LinearConstraint[](x_2_3_0+x_2_3_1+x_2_4_0+x_2_4_1,LE,x_0_2_0+x_0_2_1+x_1_2_0+x_1_2_1),
 docplex.mp.LinearConstraint[](x_3_4_0+x_3_4_1,LE,x_0_3_0+x_0_3_1+x_1_3_0+x_1_3_1+x_2_3_0+x_2_3_1)]

In [5428]:
mdl.add_constraints( C[i] - M*(1-x[i,j,k]) <= C[j] - Q[k] - Cout[i][j] 
                    for i in [1] for j in S[i]  for k in range(K) )

[docplex.mp.LinearConstraint[](2000x_1_2_0+C_1-2000,LE,C_2-65),
 docplex.mp.LinearConstraint[](2000x_1_2_1+C_1-2000,LE,C_2-65),
 docplex.mp.LinearConstraint[](2000x_1_4_0+C_1-2000,LE,C_4-51),
 docplex.mp.LinearConstraint[](2000x_1_4_1+C_1-2000,LE,C_4-51)]

In [5414]:
# mdl.add_constraints( C[i] - M*(1-x[i,j,k]) <= C[j] - (Q[k] + Cout[i][j]) 
#                     for i in [1] for j in S[i] if j!= N-1 for k in range(K) )

In [5415]:
mdl.add_constraints( C[i] - M*(1-x[i,j,k]) <= C[j]  - Cout[i][j] for i in [0] for j in S[i] for k in range(K) )

[docplex.mp.LinearConstraint[](2000x_0_1_0+C_0-2000,LE,C_1-37),
 docplex.mp.LinearConstraint[](2000x_0_1_1+C_0-2000,LE,C_1-37),
 docplex.mp.LinearConstraint[](2000x_0_2_0+C_0-2000,LE,C_2-37),
 docplex.mp.LinearConstraint[](2000x_0_2_1+C_0-2000,LE,C_2-37),
 docplex.mp.LinearConstraint[](2000x_0_3_0+C_0-2000,LE,C_3-37),
 docplex.mp.LinearConstraint[](2000x_0_3_1+C_0-2000,LE,C_3-37),
 docplex.mp.LinearConstraint[](2000x_0_4_0+C_0-2000,LE,C_4),
 docplex.mp.LinearConstraint[](2000x_0_4_1+C_0-2000,LE,C_4)]

In [5416]:
mdl.add_constraints( C[i] - mdl.sum( Q[k]*x[i,j,k] for j in S[i] for k in range(K) ) >= A[i-1] for i in L )

[docplex.mp.LinearConstraint[](-15x_1_2_0-15x_1_2_1-15x_1_4_0-15x_1_4_1+C_1,GE,280),
 docplex.mp.LinearConstraint[](-15x_2_3_0-15x_2_3_1-15x_2_4_0-15x_2_4_1+C_2,GE,280),
 docplex.mp.LinearConstraint[](-15x_3_4_0-15x_3_4_1+C_3,GE,280)]

In [5417]:
mdl.add_constraints( C[i+1] - mdl.sum(Q[k]*x[i+1,j,k] for j in S[i+1] for k in range(K) ) -C[i] <= TL for i in Less )

[docplex.mp.LinearConstraint[](-15x_2_3_0-15x_2_3_1-15x_2_4_0-15x_2_4_1-C_1+C_2,LE,5),
 docplex.mp.LinearConstraint[](-15x_3_4_0-15x_3_4_1-C_2+C_3,LE,5)]

In [5418]:
mdl.add_constraints( C[i+1] >= C[i] + mdl.sum(Q[k]*x[i,j,k] for j in S[i] for k in range(K) )   for i in Less )

[docplex.mp.LinearConstraint[](C_2,GE,15x_1_2_0+15x_1_2_1+15x_1_4_0+15x_1_4_1+C_1),
 docplex.mp.LinearConstraint[](C_3,GE,15x_2_3_0+15x_2_3_1+15x_2_4_0+15x_2_4_1+C_2)]

In [5419]:
mdl.add_constraints( A[i-1] <= C[i] for i in L )

[docplex.mp.LinearConstraint[](C_1,GE,280),
 docplex.mp.LinearConstraint[](C_2,GE,280),
 docplex.mp.LinearConstraint[](C_3,GE,280)]

In [5420]:
mdl.add_constraints( B[i-1] >= C[i] for i in L )

[docplex.mp.LinearConstraint[](C_1,LE,380),
 docplex.mp.LinearConstraint[](C_2,LE,380),
 docplex.mp.LinearConstraint[](C_3,LE,380)]

In [5421]:
mdl.add_constraints(Z[0] >= C[i] for i in L )

[docplex.mp.LinearConstraint[](Z_0,GE,C_1),
 docplex.mp.LinearConstraint[](Z_0,GE,C_2),
 docplex.mp.LinearConstraint[](Z_0,GE,C_3)]

In [5422]:
# mdl.minimize(Z[0])
mdl.maximize(y)

COnstraints

In [5423]:
mdl.print_information()

Model: scheduling
 - number of variables: 61
   - binary=51, integer=0, continuous=10
 - number of constraints: 48
   - linear=48
 - parameters: defaults
 - problem type is: MILP


In [5424]:
# mdl.add_constraints([x[0,2,1]==1])

### Import the data

In [5425]:
sol=mdl.solve()

In [5426]:
# print(sol.get_solver_log())
mdl.print_solution()

y: 1
  x_0_1_1=1
  x_0_2_0=1
  x_1_4_1=1
  x_2_3_0=1
  x_3_4_0=1
  y=1
  C_1=295.000
  C_2=310.000
  C_3=325.000
  C_4=1440.000
  Z_0=1440.000


In [5427]:
mdl.report()


* model scheduling solved with objective = 1
